In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision.models as models

In [58]:
import torch
import torch.autograd as autograd
import torch.nn as nn

class Forward(torch.autograd.Function):
    def forward(ctx, hidden_state, feed_forward):
        ctx.hidden_state = hidden_state
        return 2*feed_forward

    def backward(ctx, grad_forward):
        print(ctx.hidden_state)
        return ctx.hidden_state,grad_forward

class Backward(torch.autograd.Function):
    def forward(ctx, hidden_state, feed_back):
        ctx.hidden_state = hidden_state
        return feed_back

    def backward(ctx, grad_back):
        print(ctx.hidden_state)
        return ctx.hidden_state,grad_back

class RF(nn.Module):
    def __init__(self, hidden_size, feature_size):
        super(RF, self).__init__()
        self.hidden_size = hidden_size
        
        self.cell = nn.GRUCell(input_size=feature_size, hidden_size=hidden_size)
        
    def reset_hidden(self):
        self.hidden_state = torch.zeros((self.batch_size, self.hidden_size))
        
    def forward(self, feed_forward, feed_back):
        self.batch_size = feed_forward.size(1)
        self.reset_hidden()
        new_states = []
        for token in feed_forward.split(1):
            new_state = self.cell(token.view(1,2), self.hidden_state)
        
        return Forward()(torch.ones((self.batch_size,self.hidden_size)), feed_forward)

mod = RF(10,2)

y = torch.tensor([[[1,2]],[[1,2]]],requires_grad=True).type(torch.float) #autograd.Variable(torch.randn(1), requires_grad=True)
test_feedback = torch.tensor([[1],[2]]).type(torch.float)
z = mod(y,test_feedback)
print(z)
torch.sum(z[0]).backward()

tensor([[[ 2.,  4.]],

        [[ 2.,  4.]]])
